# feature engineering 정형 변수 생성

- reference code : 
https://www.kaggle.com/paulantoine/light-gbm-benchmark-0-3692

In [1]:
import warnings
warnings.filterwarnings("ignore")

import re
import numpy as np
import pandas as pd
import torch

In [2]:
IDIR = '/home/jovyan/work/old_nas/workspace/2_shared/TDI_BASD/02_Data/Instacart/01_og_data/'

## data load

In [3]:
print('loading prior')
priors = pd.read_csv(IDIR + 'order_products__prior.csv', 
                     dtype={'order_id': np.int32,
                            'product_id': np.uint16,
                            'add_to_cart_order': np.int16,
                            'reordered': np.int8
                           })
print('loading train')
train = pd.read_csv(IDIR + 'order_products__train.csv',
                    dtype={'order_id': np.int32,
                           'product_id': np.uint16,
                           'add_to_cart_order': np.int16,
                           'reordered': np.int8
                          })
print('loading orders')
orders = pd.read_csv(IDIR + 'orders.csv', 
                     dtype={'order_id': np.int32,
                            'user_id': np.int32,
                            'eval_set': 'category',
                            'order_number': np.int16,
                            'order_dow': np.int8,
                            'order_hour_of_day': np.int8,
                            'days_since_prior_order': np.float32
                           })
print('loading products')
products = pd.read_csv(IDIR + 'products.csv', 
                       dtype={'product_id': np.uint16,
                              'order_id': np.int32,
                              'aisle_id': np.uint8,
                              'department_id': np.uint8},
                       usecols=['product_id', 'product_name', 'aisle_id', 'department_id'])

print('priors {}: {}'.format(priors.shape, ', '.join(priors.columns)))
print('orders {}: {}'.format(orders.shape, ', '.join(orders.columns)))
print('train {}: {}'.format(train.shape, ', '.join(train.columns)))

loading prior
loading train
loading orders
loading products
priors (32434489, 4): order_id, product_id, add_to_cart_order, reordered
orders (3421083, 7): order_id, user_id, eval_set, order_number, order_dow, order_hour_of_day, days_since_prior_order
train (1384617, 4): order_id, product_id, add_to_cart_order, reordered


## product 관련 featuers
- orders : product별로 구매된 건수
- reorders : product별로 재구매된 건수
- reorder_rate : product별 재구매 비율
- organic_yn : "organic" 의 단어가 상품 이름 안에 포함되어있는지 여부
- gluten_free_yn : "gluten_free"의 단어가 상품 이름 안에 포함되어 있는지 여부
- ~~reorder_interval : 상품별 평균적인 재구매 interval~~
- ~~one_shot_cnt : 일회성 구매 건수~~

In [4]:
prods = pd.DataFrame()
prods['orders'] = priors.groupby('product_id').size().astype(np.int32)
prods['reorders'] = priors.groupby('product_id').reordered.sum().astype(np.float32)
prods['reorder_rate'] = (prods.reorders / prods.orders).astype(np.float32)
products = products.join(prods, on='product_id')
products['organic_yn'] = np.where(products.product_name.str.lower().str.contains('organic'), 1, 0)
products['gluten_free_yn'] = np.where(products.product_name.str.lower().str.contains('gluten free'), 1, 0)
products.set_index('product_id', drop=False, inplace=True)

del prods
# order data와 join
priors = priors.merge(orders, how = 'left', on='order_id')

## user 관련 features
- average_days_between_orders : user별로 방문 기간 평균
- nb_orders : user별 order 횟수
- total_items : user별 구매한 상품 개수(중복 포함)
- all_products : user별 구매한 상품 종류
- total_distnct_items : user별 구매한 상품 종류 개수
- average_basket : user별 장바구니에 담은 상품 평균 개수

In [5]:
usr = pd.DataFrame()
usr['average_days_between_orders'] = orders.groupby('user_id')['days_since_prior_order'].mean().astype(np.float32)
usr['nb_orders'] = orders.groupby('user_id').size().astype(np.int16)

users = pd.DataFrame()
users['total_items'] = priors.groupby('user_id').size().astype(np.int16)
users['all_products'] = priors.groupby('user_id')['product_id'].apply(set)
users['total_distinct_items'] = (users.all_products.map(len)).astype(np.int16)

users = users.join(usr)
users['average_basket'] = (users.total_items / users.nb_orders).astype(np.float32)

del usr

## user X product 관련 변수
- 아래 코드는 base를 위한 변수이며, 그 다음 변수를 붙일 때 변수 사용

In [6]:
userXproduct = pd.DataFrame()
userXproduct[['user_id', 'product_id', 'last_order_id']] = priors.sort_values(['user_id','product_id','order_number']
                                                                             ).drop_duplicates(['user_id', 'product_id'], keep = 'last'
                                                                                              )[['user_id', 'product_id', 'order_id']].reset_index(drop = True)
userXproduct['nb_orders'] = priors.groupby(['user_id', 'product_id']).size().astype(np.int16).values
userXproduct['sum_pos_in_cart'] = priors.groupby(['user_id', 'product_id']).add_to_cart_order.sum().astype(np.int16).values

## feature 붙이기

In [7]:
eval_order = orders[orders.eval_set != 'prior'].set_index('user_id', drop=False)
orders.set_index('order_id', drop = False, inplace = True)

In [8]:
df = userXproduct[['user_id', 'product_id']]
df['order_id'] = df.user_id.map(eval_order.order_id)
df['eval_set'] = df.user_id.map(eval_order.eval_set)

## user relative features
df['user_total_orders'] = df.user_id.map(users.nb_orders)
df['user_total_items'] = df.user_id.map(users.total_items)
df['total_distinct_items'] = df.user_id.map(users.total_distinct_items)
df['user_average_days_between_orders'] = df.user_id.map(users.average_days_between_orders)
df['user_average_basket'] =  df.user_id.map(users.average_basket)

# order relative features
df['order_dow'] = df.user_id.map(eval_order.order_dow)
df['order_hour_of_day'] = df.user_id.map(eval_order.order_hour_of_day)
df['days_since_prior_order'] = df.user_id.map(eval_order.days_since_prior_order)
df['days_since_ratio'] = df.days_since_prior_order / df.user_average_days_between_orders

# product relative features
df['aisle_id'] = df.product_id.map(products.aisle_id)
df['department_id'] = df.product_id.map(products.department_id)
df['product_orders'] = df.product_id.map(products.orders).astype(np.int32)
df['product_reorders'] = df.product_id.map(products.reorders)
df['product_reorder_rate'] = df.product_id.map(products.reorder_rate)

# user X product relative features
df['UP_orders'] = userXproduct.nb_orders
df['UP_orders_ratio'] = (df.UP_orders / df.user_total_orders).astype(np.float32)
df['UP_last_order_id'] = userXproduct.last_order_id
df['UP_average_pos_in_cart'] = (userXproduct.sum_pos_in_cart / df.UP_orders).astype(np.float32)
df['UP_reorder_rate'] = (df.UP_orders / df.user_total_orders).astype(np.float32)
df['UP_orders_since_last'] = df.user_total_orders - df.UP_last_order_id.map(orders.order_number)
df['UP_delta_hour_vs_last'] = abs(df.order_hour_of_day - df.UP_last_order_id.map(orders.order_hour_of_day)).map(lambda x: min(x, 24-x)).astype(np.int8)
df.drop(['UP_last_order_id'], axis=1, inplace=True)

In [9]:
label = train[['order_id', 'product_id']]
label['label'] = 1

df = pd.merge(df, label, how = 'left', on = ['order_id', 'product_id'])
df['label'] = np.where(df.eval_set == 'test', -1, 
                       np.where(df.label.isnull(), 0, 1))

In [10]:
df

,user_id,product_id,order_id,eval_set,user_total_orders,user_total_items,total_distinct_items,user_average_days_between_orders,user_average_basket,order_dow,...,product_orders,product_reorders,product_reorder_rate,UP_orders,UP_orders_ratio,UP_average_pos_in_cart,UP_reorder_rate,UP_orders_since_last,UP_delta_hour_vs_last,label
0,1,196,1187899,train,11,59,18,19.000000,5.363636,4,...,35791,27791.0,0.776480,10,0.909091,1.400000,0.909091,1,0,1
1,1,10258,1187899,train,11,59,18,19.000000,5.363636,4,...,1946,1389.0,0.713772,9,0.818182,3.333333,0.818182,1,0,1
2,1,10326,1187899,train,11,59,18,19.000000,5.363636,4,...,5526,3603.0,0.652009,1,0.090909,5.000000,0.090909,6,7,0
3,1,12427,1187899,train,11,59,18,19.000000,5.363636,4,...,6476,4797.0,0.740735,10,0.909091,3.300000,0.909091,1,0,0
4,1,13032,1187899,train,11,59,18,19.000000,5.363636,4,...,3751,2465.0,0.657158,3,0.272727,6.333333,0.272727,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13307948,206209,43961,272231,train,14,129,68,18.461538,9.214286,6,...,55371,34916.0,0.630583,3,0.214286,8.000000,0.214286,2,2,0
13307949,206209,44325,272231,train,14,129,68,18.461538,9.214286,6,...,3485,1398.0,0.401148,1,0.071429,8.000000,0.071429,7,1,0
13307950,206209,48370,272231,train,14,129,68,18.461538,9.214286,6,...,3934,2751.0,0.699288,1,0.071429,8.000000,0.071429,3,4,0
13307951,206209,48697,272231,train,14,129,68,18.461538,9.214286,6,...,9783,3499.0,0.357661,1,0.071429,6.000000,0.071429,7,1,0


In [11]:
torch.save(df, './3rd_code/pre_data/statistic_feature.torch')